# Définition

**Beautiful Soup** est une bibliothèque Python permettant d'extraire des données de fichiers HTML et XML. Il fonctionne avec votre analyseur préféré pour fournir des moyens idiomatiques de naviguer, de rechercher et de modifier l'arborescence d'analyse. Cela permet généralement aux programmeurs d'économiser des heures ou des jours de travail.

# Installation de BeautifulSoup 

In [1]:
!pip3 install beautifulsoup4

# Exemple d'utilisation

### Contexte 

Prenons le site de *seloger.com*. On veut chercher des annonces qui correspondent à un loyer de 1200€ max, sur Paris, ... . Le problème c'est qu'il y a près de 6000 annonces proposées sur 300 pages, chaque page regroupant 20 annonces.
Voici les 3 premières pages :

```
https://www.seloger.com/immobilier/locations/immo-paris-75/bien-appartement/?LISTING-LISTpg=1
https://www.seloger.com/immobilier/locations/immo-paris-75/bien-appartement/?LISTING-LISTpg=2
https://www.seloger.com/immobilier/locations/immo-paris-75/bien-appartement/?LISTING-LISTpg=3
```

Elles sont quasi identiques. Du coup pour obtenir la liste de ces 300 pages, il suffit de changer le dernier chiffre qui indique le numéro de la page. On peut le faire de la façon suivante:

In [4]:
token = 'https://www.seloger.com/immobilier/locations/immo-paris-75/bien-appartement/?LISTING-LISTpg='

def get_pages(token, nb):
    pages = []
    for i in range(1,nb+1):
        j = token + str(i)
        pages.append(j)
    return pages


pages = get_pages(token,300)

### Navigation dans un document HTML

Maintenant que l'on a une liste d'adresse web, il faut que l'on puisse y accéder.

In [6]:
# requests est une librairie HTTP python qui a pour but de simplifier les requêtes HTTP
!pip install requests

In [ ]:
import requests

for i in pages:
    response = requests.get(i)

A chaque itération, la fonction ```requests.get()``` essaie de se connecter à une adresse dans notre liste et enregistre résultat dans la variable ```response```. Si la tentative est réussie, la variable ```response``` contiendra la valeur ```<Response [200]>``` ainsi que le code source de la page à laquelle on souhaite accéder. Utilisez ```response.text``` pour l’afficher.

Les pages web sont écrites en HTML. Le code HTML contient des balises qui aident les navigateurs à afficher correctement le contenu de la page. Le plus souvent les informations que nous souhaitons extraire se trouvent au milieu des balises. Voici un exemple :

```
<!DOCTYPE html>  
<html>  
    <head>
    </head>
    <body>
        <h1> Jolie studio dans le 10ème</h1>
        <p class="promo"> 1100 euros charges comprises</p>
        <p> Tel: 06 82 23 21 </p> 
    <body>
</html>
```

Le titre de l’annonce se trouve entre les balises ```<h1>```…```</h1>```. Le prix de l’appartement et le numéro de téléphone de l’agence se situent au milieu des balises ```<p>```…```</p>```. Parfois les balises peuvent avoir une ```class``` particulière, une sorte de nom propre qui les distingue des autres balises du même type.

Le code HTML du site seloger.com est beaucoup plus complexe. On peut le visualiser à l’aide du navigateur Chrome en faisant un clique droit sur la page affichée et en choisissant “Inspecter”. La pluspart des navigateurs sont dotés d’une fonctionnalité qui permet la recherche des balises dans le code de la page à partir de mots clés.

Il s’avère que les informations que nous voulons extraire se trouvent entre les balises ```<em>```...```</em>``` :

```
<em class="agency-website" data-codeinsee="750118" data-codepostal="75018" data-idagence="46600" data-idannonce="119026673" data-idtiers="22739" data-idtypepublicationsourcecouplage="SL" data-nb_chambres="0" data-nb_photos="6" data-nb_pieces="1" data-position="3" data-prix="999 €" data-produitsvisibilite="AD:AC:BB:BX:AW" data-surface="32,2400016784668" data-typebien="1" data-typedetransaction="1"> Site web </em>
```

Pour accéder aux informations de cette balise, il faudra la retrouver dans le code HTML de notre variable ```response```. On peut le faire à l’aide de la librairie BeautifulSoup. Nous transformons d’abord ```response``` en objet BeautifulSoup et cherchons ensuite toutes les balises ```<em>```...```</em>``` à l’aide de la fonction ```find_all()```. On précise entre parenthèses que nous voulons les balises ```<em>```...```</em>``` qui portent le nom ```class="agency-website"``` :

In [ ]:
import bs4

soup = bs4.BeautifulSoup(response.text, 'html.parser')
em_box = soup.find_all("em", {"class":"agency-website"})

Nous avons presque atteint notre objectif ! Il nous reste à extraire une par une les informations qui nous intéressent. La variable ```em_box``` fonctionne comme un dictionnaire. Quand on donne une clé à un de ses éléments, par exemple, ```em_box[3]['data-prix']```, elle nous renvoie sa valeur: ```'999 €'```. Nous pouvons donc faire une boucle qui extrait toutes les valeurs qu’on désire et les range dans une table :

In [ ]:
import pandas as pd
parameters = ['data-prix','data-codepostal','data-idagence','data-idannonce','data-nb_chambres','data-nb_pieces','data-surface','data-typebien']

df_f = pd.DataFrame()

for par in parameters:
    l = []
    
    for el in em_box:
        j = el[par]
        l.append(j)
        
    l = pd.DataFrame(l, columns = [par])
    
    df_f = pd.concat([df_f,l], axis = 1)

Les deux dernières lignes de code regroupent les valeurs extraites en colonnes et joignent ces colonnes ensemble. Et voilà, nous avons réussi à scraper une page :

![r4g6ire9.bmp](https://miro.medium.com/max/2400/1*5_ogdVz7zTlrhboICW9AsA.png)

# Ne pas passer pour un robot

Maintenant il faut répéter ça pour les 299 autres pages. Le problème c'est qu'au bout de quelques itérations la fonction ```requests.get()``` renverra la variable ```response``` suivante :

```
<!DOCTYPE html>

<html>
<head>
<meta content="noindex,nofollow" name="robots"/>

```

La balise ```meta``` montre que seloger.com nous a identifié comme un robot.

Pour connaître la liste des robots autorisés sur un site, il faut lire son fichier ```robots.txt```:


In [ ]:
from urllib.request import urlopen

with urlopen("https://www.seloger.com/robots.txt") as stream:
    print(stream.read().decode("utf-8"))

Voici un extrait de ```robots.txt``` du site seloger.com :
![image_robots.txt](https://miro.medium.com/max/2400/1*xI7MNnWIa1Z3rozhJsQ_Sg.png)

On constate qu’il bannit les robots comme *Srapy*, *Zao*, *UbiCrawler*, *Nutch* et bien d’autres. Le nom du robot est précédé par ```User_agent```. C’est une sorte de carte d’identité de l’utilisateur collectée par les sites. Les robots se présentent simplement avec leur noms. Cependant, si on accède à seloger.com depuis un navigateur, le champ ```User_agent``` contiendra le nom de notre navigateur et les informations concernant notre système d’exploitation, par exemple : ```Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0```.

Quel est l’```User_agent``` de notre robot ? Essayez la commande suivante: ```requests.utils.default_user_agent()```. En l’exécutant vous allez apprendre que votre robot s’appelle ‘python-requests/2.21.0’. Il ne figure pas sur la liste des mauvais robots, mais cela ne signifie pas pour autant qu’il est le bienvenu. Le serveur de seloger.com détecte rapidement son comportement ‘robotique’ et le bloque temporairement.

À la différence des humains, les robots :

* N’utilisent pas les navigateurs et possèdent donc un mauvais ```User_agent```
* Restent très peu de temps sur une page
* Demandent plusieurs pages à la fois depuis la même adresse IP

Afin d’éviter d’être bloqué, on peut demander à notre robot de se présenter correctement, comme un humain, en précisant un bon ```User_agent```. Pour ne pas endommager le site et se comporter de façon plus “humaine”, on peut également demander à notre robot de patienter quelques secondes sur une page avant de passer à une autre. Enfin, à chaque passage, nous pouvons changer l’adresse IP du robot.

Voici comment réaliser ces ajustements. Commençons par créer un pool de proxies. On trouve plusieurs listes de proxies gratuits sur Internet (exemple: www.proxy-list.download/HTTPS). Il faudra la télécharger et l’importer l'espace de travail. On crée ensuite un **itérateur**, un objet qui nous permettra changer de proxy à chaque connexion à l’aide de la fonction ```next()```.

In [ ]:
import itertools as it

proxies = pd.read_csv('proxy_list.txt', header = None)
proxies = proxies.values.tolist()
proxies = list(it.chain.from_iterable(proxies))
proxy_pool = it.cycle(proxies)
proxy = next(proxy_pool)

Maintenant, apprenons notre robot à se présenter correctement et à être moins impatient. Pour le faire, nous utilisons les librairies ```time```,```random``` et ```fake-useragent```:

In [ ]:
import random
import time

# !pip install fake-useragent

from fake_useragent import UserAgent

ua = UserAgent()
time.sleep(random.randrange(1,5))

Par conséquent, on modifie notre fonction ```requests.get()```:

In [ ]:
response = requests.get(i,proxies={"http": proxy, "https": proxy}, headers={'User-Agent': ua.random},timeout=5)

Elle contient maintenant un ```proxy``` qui change à chaque nouvelle exécution de la ligne ```proxy = next(proxy_pool)``` et une fausse identité ```ua.random```. Nous avons ajouté également l’argument ```timeout=5``` qui signifie que chaque tentative de connexion à une page dure 5 secondes maximum.

Les proxies gratuits ne sont jamais très fiables. Afin de revenir sur une page que notre fonction ```requests.get()``` n’est pas parvenue à ouvrir à cause d’une mauvaise connexion, nous avons ajouté la condition suivante :

```
while len(pages) > 0:
    try:
       ...
       pages.remove(i)
    except:
       print("Skipping. Connnection error")
```

La boucle ```while``` répétera une tentative de connexion jusqu’à l’épuisement des pages à consulter dans notre liste.

Avec une telle requête, le serveur ne saura jamais que vous êtes un robot et vous obtiendriez toutes les informations souhaitées. Notons toutefois que seloger.com interdit le scraping dans ses conditions d’utilisation. L’exécution du code de ce tutoriel relève donc entièrement de votre responsabilité !

# Conclusion

Les techniques de scraping facilitent la collecte et le traitement des données dispersées sur intternet. Pour le réaliser il faut:

* Créer une liste des pages web à parcourir.
* Localiser l’information qui vous intéresse dans le code source de ces pages. Avoir des bases en HTML vous facilitera cette étape du travail.
* Une fois que vous avez identifié les balises à extraire, vous pouvez créer une boucle qui répète la même opération pour plusieurs pages.
* Si le site vous bloque, changez d’User_agent et utilisez un proxy.

# Code complet


In [ ]:
import pandas as pd
import time
import bs4
import random
import requests
# !pip install fake-useragent
from fake_useragent import UserAgent
import itertools as it

token = 'https://www.seloger.com/immobilier/locations/immo-paris-75/bien-appartement/?LISTING-LISTpg='

def get_pages(token, nb):
    pages = []
    for i in range(1,nb+1):
        j = token + str(i)
        pages.append(j)
    return pages

pages = get_pages(token,295)

# https://www.proxy-list.download/HTTPS
proxies = pd.read_csv('proxy_list.txt', header = None)
proxies = proxies.values.tolist()
proxies = list(it.chain.from_iterable(proxies))

def get_data(pages,proxies):
    
    df = pd.DataFrame()
    parameters = ['data-prix','data-codepostal','data-idagence','data-idannonce','data-nb_chambres','data-nb_pieces','data-surface','data-typebien']
    ua = UserAgent()
    proxy_pool = it.cycle(proxies)
    
    while len(pages) > 0:
        for i in pages:
        # on lit les pages une par une et on initialise une table vide pour ranger les données d'une page     
            df_f = pd.DataFrame()
        # itération dans un liste de proxies    
            proxy = next(proxy_pool)
        # essai d'ouverture d'une page   
            try:
                response = requests.get(i,proxies={"http": proxy, "https": proxy}, headers={'User-Agent': ua.random},timeout=5)
                time.sleep(random.randrange(1,5))
        # lecture du code html et la recherche des balises <em>
                soup = bs4.BeautifulSoup(response.text, 'html.parser')
                em_box = soup.find_all("em", {"class":"agency-website"})
        # extraction des données        
                for par in parameters:
                    l = []
                    for el in em_box:
                        j = el[par]
                        l.append(j)
                    l = pd.DataFrame(l, columns = [par])
                    df_f = pd.concat([df_f,l], axis = 1)
                df = df.append(df_f, ignore_index=True)
                pages.remove(i)
                print(df.shape)
            except:
                print("Skipping. Connnection error")
                
    return df

data = get_data(pages,proxies)